# Gumbel Type II

The classical notion of a random effect has proven to be a convenient way to introduce association and unobserved heterogeneity. We will now derive the PC prior for the precision parameter $\tau$ for a Gaussian random effect $x$, where $x \sim N(0, \tau^{-1}R^{-1})$, with $R \ge 0$ known. In allowing R to be indefinite, this derivation also includes popular intrinsic models such as CAR and thinplate spline models. The natural base model is the absence of random effects, which
corresponds to $\tau = \infty$. In the rank deficient case, the natural base model is that the effect belongs to the nullspace of $R$, which also corresponds to $\tau = \infty$. This base model leads to a useful negative result.

The PC prior for $\tau$ is, except for in the specification of $\lambda$, independent of $R$ and expressed below 

$$
\pi(\tau) = \frac{\lambda}{2} \tau^{-3/2} \exp(-\lambda \tau^{-1/2})
$$


In [2]:
import numpy as np
import pystan as ps
import stan_utility

import matplotlib.pyplot as plt
import seaborn as sns
import arviz as az

import pickle

For that, we consider a very simple model. 

In [24]:
compiled = False
#compiled = True

if compiled: 
    sm = pickle.load(open('../models/extra/gumbel_dist.pkl', 'rb'))
else: 
    sm = ps.StanModel(file = '../models/extra/gumbel_dist.stan')
    with open('../models/extra/gumbel_dist.pkl', 'wb') as f:
        pickle.dump(sm, f)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_85cc2bb3b4778c5271b0f49e80ca1662 NOW.


In [25]:
print(sm.model_code)

functions {
  real gumbel_type2_lpdf(real tau, real lambda){
    return -(3/2 * log(tau) + lambda / sqrt(tau)); 
  }
}
data {
   int<lower = 0> n;
   real<lower = 0> lambda;
   cov_matrix[n] c; 
   vector[n] y;  
}
parameters {
   real<lower = 0> tau;
}
model {
   tau ~ gumbel_type2(lambda);
   y ~ multi_normal(rep_vector(0, n), (1/sqrt(tau)) * c);
}


In [26]:
cov_matrix = np.array([ 
    [2, 1, 1, 0, 0], 
    [1, 2, 1, 0, 0], 
    [1, 1, 4, 0, 1], 
    [0, 0, 0, 2, 1], 
    [0, 0, 1, 1, 3]
])

tau = 3

y = np.random.multivariate_normal(mean = np.zeros(5), 
                                  cov = (1/np.sqrt(tau)) * cov_matrix)

data = {
    'n': 5,
    'lambda': 1, 
    'c': cov_matrix, 
    'y': y
}

In [29]:
fit = sm.sampling(data = data)

In [30]:
fit

Inference for Stan model: anon_model_85cc2bb3b4778c5271b0f49e80ca1662.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

       mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
tau   22.48    0.62  26.74   1.06    5.8  13.66  28.67  97.16   1882    1.0
lp__  -1.83    0.02   0.67  -3.75  -2.01  -1.56  -1.38  -1.33   1599    1.0

Samples were drawn using NUTS at Thu Oct  7 00:10:30 2021.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).